Importing Basic Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Reading Files

In [2]:
box = pd.read_csv(r'C:\Users\Admin\Desktop\Datathon\Problem 1\boxes.csv')
purchase= pd.read_csv(r'C:\Users\Admin\Desktop\Datathon\Problem 1\purchase.csv')

Dropping duplicates, changing datatypes properly

In [3]:
purchase.dropna(subset=['BOX_ID', 'BOX_COUNT'], inplace=True)

# Convert PURCHASE_DATE to datetime
purchase['PURCHASE_DATE'] = pd.to_datetime(purchase['PURCHASE_DATE'], format='%d/%m/%Y')

# Convert BOX_ID to integer
purchase['BOX_ID'] = purchase['BOX_ID'].astype('int32')

# Convert BOX_COUNT to integer
purchase['BOX_COUNT'] = purchase['BOX_COUNT'].astype('int32')

box['BOX_ID'] = box['BOX_ID'].astype('int32')

box = box.drop_duplicates()
purchase = purchase.drop_duplicates()

In [4]:
df = pd.merge(box, purchase, how='inner')
df.head()

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96,2019-02-04,2C88D36D1FC,1
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2BF011BDB38,1
2,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2CA0EE8F2B3,1
3,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2C623730B09,1
4,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2CA6CE1054F,1


In [5]:
df = df.drop(columns=['QUALITY', 'DELIVERY_OPTION', 'MILK', 'MEAT', 'UNIT_PRICE'])
df.head()

,BOX_ID,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT
0,1,2019-02-04,2C88D36D1FC,1
1,2,2019-01-02,2BF011BDB38,1
2,2,2019-01-02,2CA0EE8F2B3,1
3,2,2019-01-02,2C623730B09,1
4,2,2019-01-02,2CA6CE1054F,1


In [6]:
df['BOX_COUNT'].value_counts()

BOX_COUNT
1     2453301
2        2272
3          83
4          28
6          17
5          10
7           3
9           2
19          2
11          2
8           1
10          1
13          1
Name: count, dtype: int64

In [7]:
# Drop rows where BOX_COUNT is more than 1
df = df[df['BOX_COUNT'] <= 1]

In [8]:
# Find the most recent purchase date in the dataset
max_purchase_date = df['PURCHASE_DATE'].max()

# Calculate the last purchase date for each MAGIC_KEY
last_purchase_date = df.groupby('MAGIC_KEY')['PURCHASE_DATE'].max()

# Days since last purchase from the max purchase date
last_purchase_date = max_purchase_date - last_purchase_date

# To find the average time window between purchases for each MAGIC_KEY
def calculate_average_time(group):
    if len(group) > 1:
        return (group.max() - group.min()) / (len(group) - 1)
    else:
        return pd.Timedelta('0 days')

average_purchase_window = df.groupby('MAGIC_KEY')['PURCHASE_DATE'].apply(calculate_average_time)

# Combine all the results into a single DataFrame
result = pd.DataFrame({
    'Last Purchase Date': last_purchase_date,
    'Days Since Last Purchase': last_purchase_date.dt.days,
    'Average Purchase Window (days)': average_purchase_window.dt.days
}).reset_index()

In [11]:
result = result.drop(columns=['Last Purchase Date'])

,MAGIC_KEY,Days Since Last Purchase,Average Purchase Window (days)
0,249670911D8,26,93
1,249751FC4DD,139,0
2,24978027606,132,0
3,24979164422,90,0
4,2497B8B4FDA,119,29


In [12]:
result.sample(10)

,MAGIC_KEY,Days Since Last Purchase,Average Purchase Window (days)
173709,2909924B2A6,142,0
859263,2C764545CB8,9,31
1262263,2CFF2D57481,56,0
214331,291B6332FDF,14,19
976552,2CA0397CC96,136,0
239777,2936B2BAC40,72,70
420095,2BD866FBABA,4,0
584944,2C0892A0128,60,0
981081,2CA1D5FE18E,90,30
791656,2C5874D89F9,70,0


In [13]:
# Calculate "will purchase" with modified logic for 0-day average window
result['will purchase'] = (((result['Average Purchase Window (days)'] - result['Days Since Last Purchase']) <= 15) & (result['Average Purchase Window (days)'] > 0)) | ((result['Days Since Last Purchase'] >= result['Average Purchase Window (days)']) & (result['Average Purchase Window (days)'] > 0))

In [14]:
result.sample(10)

,MAGIC_KEY,Days Since Last Purchase,Average Purchase Window (days),will purchase
720050,2C37EF2FD0C,8,0,False
958017,2C9A8572BBE,17,0,False
1145089,2CD704317B1,31,0,False
380648,2BCE03241AC,2,29,False
543127,2BFABCF6A3C,67,28,True
897079,2C8515D5E39,65,47,True
1212593,2CEAD680182,65,58,True
1044294,2CB67EA0847,55,31,True
209324,29164D9A875,17,0,False
493304,2BEAC6A72FB,45,51,True


In [15]:
result['will purchase'].value_counts()

will purchase
False    981279
True     292185
Name: count, dtype: int64

In [16]:
# Convert True/False in 'will purchase' to 'Y'/'N'
result['will purchase'] = result['will purchase'].map({True: 'Y', False: 'N'})

In [17]:
result['will purchase'].value_counts()

will purchase
N    981279
Y    292185
Name: count, dtype: int64

In [19]:
result['MAGIC_KEY'].duplicated().sum()

0

In [20]:
problem = pd.read_csv(r'C:\Users\Admin\Desktop\Datathon\Problem 1\problem_1.csv')

In [21]:
merged_data = problem.merge(result, on='MAGIC_KEY', how='left')

In [22]:
merged_data.head()

,MAGIC_KEY,Days Since Last Purchase,Average Purchase Window (days),will purchase
0,28D5BB06356,2.0,32.0,N
1,293BEAB4E98,20.0,41.0,N
2,2962EE8065C,11.0,26.0,Y
3,2957BE29EA9,56.0,29.0,Y
4,28E351A0745,34.0,29.0,Y


In [23]:
merged_data = merged_data.drop(columns=['Days Since Last Purchase', 'Average Purchase Window (days)'])

In [25]:
# Rename 'will purchase' column to 'PURCHASE'
merged_data.rename(columns={'will purchase': 'PURCHASE'}, inplace=True)

In [26]:
merged_data.head()

,MAGIC_KEY,PURCHASE
0,28D5BB06356,N
1,293BEAB4E98,N
2,2962EE8065C,Y
3,2957BE29EA9,Y
4,28E351A0745,Y


In [27]:
# Save the DataFrame
merged_data.to_csv('submission.csv', index=False)